In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
from tqdm import tqdm

import numpy as np

from low_energy.lowE_electrons import make_interpolator
from low_energy.lowE_deposition import compute_fs
from utils import load_dict

sys.path.append(os.environ['DH_DIR'])
from darkhistory.config import load_data
from darkhistory.main import get_elec_cooling_data
from   darkhistory.spec.spectrum import Spectrum
import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from   darkhistory.electrons import positronium as pos
from   darkhistory.electrons.elec_cooling import get_elec_cooling_tf

## Config

In [2]:
run_name = '230629'
tf_type = 'phot' # {'phot', 'elec'}
include_f_xray = True
xray_eng_range = (1e2, 1e4) # [eV]
use_tqdm = True
verbose = 0 # {0, 1}
stop_after_n = np.inf

abscs = load_dict(f"../data/abscissas/abscs_{run_name}.h5")
inj_abscs = abscs['photE'] if tf_type == 'phot' else abscs['elecEk'] + phys.me
MEDEA_dir = '../data/MEDEA'
data_dir = f'../data/tf/{run_name}/{tf_type}'
save_dir = f'../data/tf/{run_name}/{tf_type}'
os.makedirs(save_dir, exist_ok=True)

In [3]:
abscs['dep_c']

array(['H ion', 'He ion', 'exc', 'heat', 'cont', 'xray'], dtype=object)

## Load

In [4]:
os.listdir(data_dir)

['lep_tf_rxneo.npy',
 'highengdep_interp.raw',
 'lowengelec_tf_interp.raw',
 'phot_dep.h5',
 'lowerbound_rxneo.npy',
 'phot_depgv.npy',
 'highengphot_tf_interp.raw',
 'hed_tf_rxneo.npy',
 'lowengphot_tf_interp.raw',
 'cmbloss_rxneo.npy',
 'phot_tfgv.npy',
 'phot_phot.h5',
 'CMB_engloss_interp.raw',
 'phot_scat.h5',
 'hep_tf_rxneo.npy',
 'phot_prop.h5',
 '.ipynb_checkpoints',
 'ionhist_output',
 'phot_prop_diag.h5',
 'lee_tf_rxneo.npy']

In [7]:
print('Loading tf: ', end=' ', flush=True)
hep_tfgv = np.load(f'{data_dir}/hep_tf_rxneo.npy')
print('hep', end=' ', flush=True)
lep_tfgv = np.load(f'{data_dir}/lep_tf_rxneo.npy')
print('lep', end=' ', flush=True)
lee_tfgv = np.load(f'{data_dir}/lee_tf_rxneo.npy')
print('lee', end=' ', flush=True)
hed_tfgv = np.load(f'{data_dir}/hed_tf_rxneo.npy')
print('hed', end=' ', flush=True)
cmbloss_gv = np.load(f'{data_dir}/cmbloss_rxneo.npy')
print('cmb', end='.', flush=True)

Loading tf:  hep lep lee hed cmb.

## Debug

In [53]:
tfgv = np.zeros_like(hep_tfgv) # in: elec/phot, out: phot
depgv = np.zeros(
    hed_tfgv.shape[:-1] + (len(abscs['dep_c']),)
) # channels: {H ionization, He ionization, excitation, heat, continuum, xray}

In [54]:
dlnz = abscs['dlnz']
print(dlnz)

0.009950330853168092


In [55]:
MEDEA_interp = make_interpolator(prefix=MEDEA_dir)

In [56]:
n_run = -1

In [57]:
i_rs = 8
rs = abscs['rs'][i_rs]
print(rs)

38.71318413405634


In [58]:
dt = dlnz / phys.hubble(rs)
cmb_un = spectools.discretize(abscs['photE'], phys.CMB_spec, phys.TCMB(rs))
cmb_un_E = cmb_un.toteng()

In [59]:
i_x = 3
x = abscs['x'][i_x]
print(x)

0.01


In [9]:
nBs = 1
nBs_f_left = (abscs['nBs'][6] - 1) / (abscs['nBs'][6] - abscs['nBs'][5])
nBs_f_right = (1 - abscs['nBs'][5]) / (abscs['nBs'][6] - abscs['nBs'][5])
nBs_f_left, nBs_f_right

(0.0967741935483871, 0.9032258064516129)

In [61]:
i = 300

In [62]:
cmbloss_gv_interp = cmbloss_gv[i_rs, i_x, 5][i]*nBs_f_left + cmbloss_gv[i_rs, i_x, 6][i]*nBs_f_right

In [63]:
cmb_E = cmbloss_gv_interp * dt

In [64]:
hep_tfgv_interp = (-cmb_E/cmb_un_E) * cmb_un.N + hep_tfgv[i_rs, i_x, 5][i]*nBs_f_left + hep_tfgv[i_rs, i_x, 6][i]*nBs_f_right

In [65]:
for i in range(len(inj_abscs)):
    if lep_tfgv[i_rs, i_x, 5][i][i] > 1e-40:
        break
    lep_tfgv[i_rs, i_x, 5][i][i] = 1

In [66]:
for i in range(len(inj_abscs)):
    if lep_tfgv[i_rs, i_x, 6][i][i] > 1e-40:
        break
    lep_tfgv[i_rs, i_x, 6][i][i] = 1

In [67]:
lep_tfgv_interp = lep_tfgv[i_rs, i_x, 5][300]*nBs_f_left + lep_tfgv[i_rs, i_x, 6][300]*nBs_f_right

In [68]:
i_injE = 300
hep_spec_N = hep_tfgv_interp
lep_spec_N = lep_tfgv_interp
lee_spec_N = lee_tfgv[i_rs, i_x, 5, i_injE]*nBs_f_left + lee_tfgv[i_rs, i_x, 6, i_injE]*nBs_f_right
hed_arr    = hed_tfgv[i_rs, i_x, 5, i_injE]*nBs_f_left + hed_tfgv[i_rs, i_x, 6, i_injE]*nBs_f_right

In [ ]:
# darkhistory
highengphot_spec_at_rs.toteng() 97.94601294262714
lowengphot_spec_at_rs.toteng() 0.0
lowengelec_spec_at_rs.toteng() 0.009064424094655703
highengdep_at_rs [ 1.20416521e-16  9.49196961e-17  3.18061073e-17 -6.01312711e-19]

In [73]:
x_vec_for_f = np.array( [1-x, phys.chi*(1-x), phys.chi*x] )
print(x_vec_for_f)

[9.90000000e-01 8.03145695e-02 8.11258278e-04]


In [75]:
injE = abscs['photE'][300]

In [76]:
nBs_ref = 1
dE_dVdt_inj = injE * phys.nB * nBs_ref * rs**3 / dt # [eV/cm^3 s]

In [78]:
lep_prop_spec_N = lep_spec_N * (abscs['photE'] < 10.2)

In [80]:
f_lep_prop = np.dot(abscs['photE'], lep_prop_spec_N) / injE

In [82]:
phot_spec_N = hep_spec_N + lep_prop_spec_N
f_prop = np.dot(abscs['photE'], phot_spec_N) / injE

In [83]:
lowengelec_spec_at_rs = Spectrum(abscs['elecEk'], lee_spec_N, spec_type='N')
lowengelec_spec_at_rs.rs = rs

lowengphot_spec_at_rs = Spectrum(abscs['photE'], lep_spec_N, spec_type='N')
lowengphot_spec_at_rs.rs = rs

highengdep_at_rs = hed_arr

In [90]:
phys.chi*0.99

0.08031456953642384

In [94]:
print(np.dot(r*hep_spec_N, abscs['photE']))
print(np.dot(r*lep_spec_N, abscs['photE']))
print(np.dot(r*lee_spec_N, abscs['elecEk'])) # 0.009064424094655703
r*hed_arr # [ 1.20416521e-16  9.49196961e-17  3.18061073e-17 -6.01312711e-19]

98.8932046183508
0.0
0.009064424094655701


array([ 1.20416521e-16,  9.49196961e-17,  3.18061073e-17, -6.01312711e-19])

In [88]:
x_vec_for_f # x_vec_for_f [9.90000000e-01 8.03145695e-02 8.11258278e-04]

array([9.90000000e-01, 8.03145695e-02, 8.11258278e-04])

In [92]:
dE_dVdt_inj*r # rate_func_eng_unclustered(rs) 4.2993000138112976e-14

4.2993000138117e-14

In [95]:
dt # dt 33533238902782.5

33533238902782.5

In [ ]:
method no_He
cross_check False
Main loop time: 0.051 s

In [84]:
f_low, f_high = compute_fs(
    MEDEA_interp=MEDEA_interp,
    rs=rs,
    x=x_vec_for_f,
    elec_spec=lowengelec_spec_at_rs,
    phot_spec=lowengphot_spec_at_rs,
    dE_dVdt_inj=dE_dVdt_inj,
    dt=dt,
    highengdep=highengdep_at_rs,
    cmbloss=0, # turned off in darkhistory main as well
    method='no_He',
    cross_check=False,
    ion_old=False
)

In [85]:
f_low #[1.70541817e-05 9.59149095e-07 1.43085440e-05 5.30452476e-05 6.27643362e-06]

array([1.70541817e-05, 9.59149095e-07, 1.43085440e-05, 5.30452476e-05,
       6.27643362e-06])

In [86]:
f_high #[ 4.08247276e-05  0.00000000e+00  3.21805571e-05  1.07832020e-05 -2.03862621e-07]

array([ 4.08247276e-05,  0.00000000e+00,  3.21805571e-05,  1.07832020e-05,
       -2.03862621e-07])

In [6]:

                # in DH.main: (dN_inj/dB) / (dE_inj  /dVdt)
                # here:       (dN_inj   ) / (dE_injdB/dVdt)
                f_low, f_high = compute_fs(
                    MEDEA_interp=MEDEA_interp,
                    rs=rs,
                    x=x_vec_for_f,
                    elec_spec=lowengelec_spec_at_rs,
                    phot_spec=lowengphot_spec_at_rs,
                    dE_dVdt_inj=dE_dVdt_inj,
                    dt=dt,
                    highengdep=highengdep_at_rs,
                    cmbloss=0, # turned off in darkhistory main as well
                    method='no_He',
                    cross_check=False,
                    ion_old=False
                )
                f_raw = f_low + f_high


                f_dep = f_raw
                f_dep[4] -= f_lep_prop # adjust for the propagating lowengphot
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Fix energy conservation (known issues)
                if tf_type == 'phot':
                    if i_injE == 153: # issue at around 13.6 eV. Adjusting H_ion.
                        f_dep[0] += 1 - f_tot
                    if i_injE in range(224, 228): # ??? issue. Adjusting hep propagating bin.
                        phot_spec_N[i_injE] += 1 - f_tot

                f_prop = np.dot(abscs['photE'], phot_spec_N) / injE
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Energy conservation
                f_dep_str = ' '.join([f'{v:.3e}' for v in f_dep])
                print_str = f'{n_run} | {i_rs} {i_x} {i_nBs} {i_injE} | f_prop={f_prop:.6f} f_dep={f_dep_str} f_tot={f_tot:.6f}'
                if np.abs(f_tot - 1.) > 1e-2:
                    print_str += ' | Energy error > 1%'
                if verbose >= 1 or np.abs(f_tot - 1.) > 1e-2:
                    print(print_str, flush=True)
                
                phot_spec_N[i_injE] += 1 - f_tot # gives all extra energy to propagating photons
                
                #==============================
                # Dependent variables (Xray)
                if include_f_xray:
                    i_xray_fm = np.searchsorted(abscs['photE'], xray_eng_range[0])
                    i_xray_to = np.searchsorted(abscs['photE'], xray_eng_range[1])
                    f_xray = np.dot(abscs['photE'][i_xray_fm:i_xray_to], phot_spec_N[i_xray_fm:i_xray_to]) / injE
                    if i_xray_fm <= i_injE and i_injE < i_xray_to:
                        f_xray -= phot_spec_N[i_injE] # ignore diagonal for now # NEED TO EXTRACT PROP
                    f_dep = np.append(f_dep, f_xray)

                #==============================
                # Populate transfer functions
                tfgv[i_rs, i_x, i_nBs, i_injE] = phot_spec_N
                depgv[i_rs, i_x, i_nBs, i_injE] = f_dep
            
            if use_tqdm:
                pbar.update()

100%|██████████| 1000/1000 [19:27<00:00,  1.17s/it]

## Loop over `rs` `x` `nBs`

In [8]:
tfgv = np.zeros_like(hep_tfgv) # in: elec/phot, out: phot
depgv = np.zeros(
    hed_tfgv.shape[:-1] + (len(abscs['dep_c']),)
) # channels: {H ionization, He ionization, excitation, heat, continuum, xray}

dlnz = abscs['dlnz']
MEDEA_interp = make_interpolator(prefix=MEDEA_dir)

n_run = -1

if use_tqdm:
    pbar = tqdm( total = len(abscs['rs'])*len(abscs['x'])*len(abscs['nBs']) )

for i_rs, rs in enumerate(abscs['rs']):
    
    if i_rs != 8:
        continue
    
    dt = dlnz / phys.hubble(rs)
            
    #==============================
    # Add cmbloss to highengphot
    cmb_un = spectools.discretize(abscs['photE'], phys.CMB_spec, phys.TCMB(rs))
    cmb_un_E = cmb_un.toteng()
    
    for i_x, x in enumerate(abscs['x']):
        if i_x != 3:
            continue
        for i_nBs, nBs in enumerate(abscs['nBs']):
            
            for i in range(len(inj_abscs)):
                cmb_E = cmbloss_gv[i_rs, i_x, i_nBs][i] * dt
                hep_tfgv[i_rs, i_x, i_nBs][i] += (-cmb_E/cmb_un_E) * cmb_un.N

            #==============================
            # Add lowengphot diagonal
            if nBs == 0: # lowengphot is 0 when nBs is 0
                raise NotImplementedError
            if tf_type == 'phot':
                for i in range(len(inj_abscs)):
                    if lep_tfgv[i_rs, i_x, i_nBs][i][i] > 1e-40:
                        break
                    lep_tfgv[i_rs, i_x, i_nBs][i][i] = 1
            
            for i_injE, injE in enumerate(inj_abscs):
                
                assert n_run <= stop_after_n
                n_run += 1

                #==============================
                # Injection
                # inject one photon at i_injE
                timer = time.time()
                
                hep_spec_N = hep_tfgv[i_rs, i_x, i_nBs, i_injE]
                lep_spec_N = lep_tfgv[i_rs, i_x, i_nBs, i_injE]
                lee_spec_N = lee_tfgv[i_rs, i_x, i_nBs, i_injE]
                hed_arr    = hed_tfgv[i_rs, i_x, i_nBs, i_injE]

                lowengelec_spec_at_rs = Spectrum(abscs['elecEk'], lee_spec_N, spec_type='N')
                lowengelec_spec_at_rs.rs = rs

                lowengphot_spec_at_rs = Spectrum(abscs['photE'], lep_spec_N, spec_type='N')
                lowengphot_spec_at_rs.rs = rs

                highengdep_at_rs = hed_arr

                #==============================
                # Compute f's
                x_vec_for_f = np.array( [1-x, phys.chi*(1-x), phys.chi*x] ) # [HI, HeI, HeII]/nH
                nBs_ref = 1
                dE_dVdt_inj = injE * phys.nB * nBs_ref * rs**3 / dt # [eV/cm^3 s]
                # in DH.main: (dN_inj/dB) / (dE_inj  /dVdt)
                # here:       (dN_inj   ) / (dE_injdB/dVdt)
                f_low, f_high = compute_fs(
                    MEDEA_interp=MEDEA_interp,
                    rs=rs,
                    x=x_vec_for_f,
                    elec_spec=lowengelec_spec_at_rs,
                    phot_spec=lowengphot_spec_at_rs,
                    dE_dVdt_inj=dE_dVdt_inj,
                    dt=dt,
                    highengdep=highengdep_at_rs,
                    cmbloss=0, # turned off in darkhistory main as well
                    method='no_He',
                    cross_check=False,
                    ion_old=False
                )
                f_raw = f_low + f_high
                
                # debug
                if i_rs == 8 and i_x == 3 and (i_nBs in [5, 6]) and i_injE == 300:
                    print(f_low)
                    print(f_high)
                    print(f_raw)

                #==============================
                # Compute tf & f values
                lep_prop_spec_N = lep_spec_N * (abscs['photE'] < 10.2)
                f_lep_prop = np.dot(abscs['photE'], lep_prop_spec_N) / injE
                phot_spec_N = hep_spec_N + lep_prop_spec_N
                f_prop = np.dot(abscs['photE'], phot_spec_N) / injE

                f_dep = f_raw
                f_dep[4] -= f_lep_prop # adjust for the propagating lowengphot
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Fix energy conservation (known issues)
                if tf_type == 'phot':
                    if i_injE == 153: # issue at around 13.6 eV. Adjusting H_ion.
                        f_dep[0] += 1 - f_tot
                    if i_injE in range(224, 228): # ??? issue. Adjusting hep propagating bin.
                        phot_spec_N[i_injE] += 1 - f_tot

                f_prop = np.dot(abscs['photE'], phot_spec_N) / injE
                f_tot = f_prop + np.sum(f_dep)

                #==============================
                # Energy conservation
                f_dep_str = ' '.join([f'{v:.3e}' for v in f_dep])
                print_str = f'{n_run} | {i_rs} {i_x} {i_nBs} {i_injE} | f_prop={f_prop:.6f} f_dep={f_dep_str} f_tot={f_tot:.6f}'
                if np.abs(f_tot - 1.) > 1e-2:
                    print_str += ' | Energy error > 1%'
                if verbose >= 1 or np.abs(f_tot - 1.) > 1e-2:
                    print(print_str, flush=True)
                
                phot_spec_N[i_injE] += 1 - f_tot # gives all extra energy to propagating photons
                
                #==============================
                # Dependent variables (Xray)
                if include_f_xray:
                    i_xray_fm = np.searchsorted(abscs['photE'], xray_eng_range[0])
                    i_xray_to = np.searchsorted(abscs['photE'], xray_eng_range[1])
                    f_xray = np.dot(abscs['photE'][i_xray_fm:i_xray_to], phot_spec_N[i_xray_fm:i_xray_to]) / injE
                    if i_xray_fm <= i_injE and i_injE < i_xray_to:
                        f_xray -= phot_spec_N[i_injE] # ignore diagonal for now # NEED TO EXTRACT PROP
                    f_dep = np.append(f_dep, f_xray)

                #==============================
                # Populate transfer functions
                tfgv[i_rs, i_x, i_nBs, i_injE] = phot_spec_N
                depgv[i_rs, i_x, i_nBs, i_injE] = f_dep
            
            if use_tqdm:
                pbar.update()


  0%|          | 5/1000 [00:05<19:17,  1.16s/it]

[1.56553207e-05 8.81128673e-07 1.31330824e-05 4.86090148e-05
 5.76069252e-06]
[ 3.73711820e-05  0.00000000e+00  2.94393096e-05  9.85873873e-06
 -1.95871606e-07]
[5.30265027e-05 8.81128673e-07 4.25723920e-05 5.84677536e-05
 5.56482091e-06]



  1%|          | 6/1000 [00:06<19:08,  1.16s/it]

[1.72040597e-05 9.67508425e-07 1.44344863e-05 5.35205583e-05
 6.33169159e-06]
[ 4.11947504e-05  0.00000000e+00  3.24742622e-05  1.08822516e-05
 -2.04718801e-07]
[5.83988101e-05 9.67508425e-07 4.69087486e-05 6.44028099e-05
 6.12697279e-06]



  1%|          | 10/1000 [00:11<18:57,  1.15s/it]

In [11]:
depgv[8, 3, 5, 300]*nBs_f_left + depgv[8, 3, 6, 300]*nBs_f_right

array([5.78789094e-05, 9.59149095e-07, 4.64891012e-05, 6.38284496e-05,
       6.07257100e-06, 6.21134076e-11])

In [15]:
z = np.load(f"{data_dir}/phot_depgv.npy")

In [18]:
z[8, 3, 5, 300], z[8, 3, 6, 300]

(array([5.30265027e-05, 8.81128673e-07, 4.25723920e-05, 5.84677536e-05,
        5.56482091e-06, 5.86350563e-11]),
 array([5.83988101e-05, 9.67508425e-07, 4.69087486e-05, 6.44028099e-05,
        6.12697279e-06, 6.24860881e-11]))

In [ ]:
from dm21cm.

In [17]:
dmcm_dep_tf = load_data(
    'phot_dep',
    prefix=os.environ['DM21CM_DATA_DIR'] + '/tf/230629/phot',
    reload=True
)

TypeError: load_data() got an unexpected keyword argument 'reload'

## Save

In [7]:
np.save(f'{save_dir}/{tf_type}_tfgv.npy', tfgv)
np.save(f'{save_dir}/{tf_type}_depgv.npy', depgv)